In [1]:
import os
from os import path
import glob
import pickle
import gc
import json
import csv

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.preprocessing import LabelEncoder

In [2]:
# 데이터 파일 목록 가져오기
file_list = glob.glob("data/*.csv")

# 삭제할 컬럼 목록
columns_to_drop = ["device_carrier", "platform", "os_version", "device_type"]

# LabelEncoder 초기화
encoders = {}

# 변환된 데이터를 저장할 CSV 파일
encoded_file = "./lwc_encoder/merged_encoded_data.csv"

# 기존 파일 불러오기
if os.path.exists(encoded_file):
    existing_df = pd.read_csv(encoded_file)
    all_columns = set(existing_df.columns)
else:
    existing_df = pd.DataFrame()
    all_columns = set()

# 기존 LabelEncoder 불러오기 (있다면)
encoder_file = "./lwc_encoder/label_encoders.pkl"
if os.path.exists(encoder_file):
    with open(encoder_file, "rb") as f:
        encoders = pickle.load(f)

# 새로운 데이터를 저장할 리스트
new_data_list = []

# 모든 파일 처리
for file in file_list:
    for chunk in pd.read_csv(file, chunksize=100000):
        chunk.drop(columns=columns_to_drop, errors="ignore", inplace=True)

        # 새로운 컬럼이 있으면 추가
        new_columns = set(chunk.columns) - all_columns
        if new_columns:
            # print(f"새로운 컬럼 발견: {new_columns}")
            all_columns.update(new_columns)

        # 인코딩할 컬럼 선택 (문자열 컬럼 중 삭제하지 않은 컬럼)
        all_string_columns = chunk.select_dtypes(include=["object"]).columns.tolist()
        columns_to_encode = [col for col in all_string_columns if col not in columns_to_drop and col != "client_event_time"]

        # 새로 발견된 문자열 컬럼에 대해 LabelEncoder 추가
        for col in columns_to_encode:
            if col not in encoders:
                encoders[col] = LabelEncoder()
                encoders[col].fit(chunk[col].astype(str).fillna(""))
            else:
                # 새로운 값이 있을 경우 기존 LabelEncoder 업데이트
                new_values = set(chunk[col].astype(str).fillna("").unique()) - set(encoders[col].classes_)
                if new_values:
                    # print(f"🔄 새로운 값 추가됨 ({col}): {new_values}")
                    updated_classes = np.append(encoders[col].classes_, list(new_values))
                    encoders[col].fit(updated_classes)  # 새 값 포함하여 다시 학습

        # Label Encoding 적용
        for col in columns_to_encode:
            chunk[col] = chunk[col].astype(str).fillna("")
            chunk[col] = encoders[col].transform(chunk[col])

        # 기존 데이터와 컬럼을 맞춤 (새로운 컬럼이 추가되면 NaN으로 채움)
        chunk = chunk.reindex(columns=all_columns, fill_value=np.nan)

        # 리스트에 추가
        new_data_list.append(chunk)

        del chunk
        gc.collect()

# 새로운 데이터프레임 병합
new_data_df = pd.concat([existing_df] + new_data_list, ignore_index=True)

# 정리된 데이터를 CSV로 저장
new_data_df.to_csv(encoded_file, index=False, sep=",", quoting=csv.QUOTE_MINIMAL)

print(f"업데이트 완료, 저장됨: {encoded_file}")

# 인코딩 맵핑 저장 (디코딩을 쉽게 하기 위해)
encoding_map = {col: {label: idx for idx, label in enumerate(encoders[col].classes_)} for col in encoders}

with open("./lwc_encoder/encoding_map.json", "w", encoding="utf-8") as f:
    json.dump(encoding_map, f, ensure_ascii=False, indent=4)

# 인코더 저장 (디코딩용)
with open(encoder_file, "wb") as f:
    pickle.dump(encoders, f)

print("인코딩 및 저장 완료 ✅")

C:\Users\lento\AppData\Local\Temp\ipykernel_29220\134091297.py:32: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=100000):
C:\Users\lento\AppData\Local\Temp\ipykernel_29220\134091297.py:32: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=100000):
C:\Users\lento\AppData\Local\Temp\ipykernel_29220\134091297.py:32: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=100000):
C:\Users\lento\AppData\Local\Temp\ipykernel_29220\134091297.py:32: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=100000):
C:\Users\lento\AppData\Local\Temp\ipykernel_29220\134091297.py:32: DtypeWarning: Columns (13) have mixed types. Specify 

업데이트 완료, 저장됨: ./lwc_encoder/merged_encoded_data.csv
인코딩 및 저장 완료 ✅
